In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
asignacion_path = 'bases/asignacion/base_asignacion_20240904_ TRABAJADA.xlsx'
df_asignacion = pd.read_excel(asignacion_path)
df_asignacion.shape

(240733, 42)

In [3]:
df_asignacion_test = df_asignacion.copy()
print(df_asignacion_test.columns)

cols_required = ['codigo', 'cont_18', 'clave', 'nombre', 'Moneda', 'Capital', 'Camp', 'CAMPAÑA', 'TIPO_CARTERA', 'AGENCIA CORRECTA', 'tipo_fondo', 'oficina', 'territorio', 'ID_VTA.']
df_asignacion_test = df_asignacion_test[cols_required]

df_asignacion_test.rename(columns={
    'codigo': 'CODIGO CENTRAL', 
    'cont_18': 'N° CONTRATO', 
    'clave': 'CARTERA', 
    'nombre': 'NOMBRE DEL CLIENTE', 
    'Moneda': 'MONEDA', 
    'Capital': 'DEUDA CAPITAL', 
    'Camp': '% CONDONACIÓN', 
    'CAMPAÑA': 'MONTO A PAGAR', 
    'TIPO_CARTERA': 'TIPO CARTERA', 
    'AGENCIA CORRECTA': 'AGENCIA DE COBRANZA', 
    'tipo_fondo': 'TIPO DE FONDO', 
    'oficina': 'OFICINA', 
    'territorio': 'TERRITORIO', 
    'ID_VTA.': 'ID_VENTA'
}, inplace=True)

df_asignacion_test = df_asignacion_test[df_asignacion_test['TIPO CARTERA'] == 'UNSECURED']
df_asignacion_test = df_asignacion_test[df_asignacion_test['ID_VENTA'].isin(['-'])]
df_asignacion_test = df_asignacion_test[~df_asignacion_test['AGENCIA DE COBRANZA'].isin(['SIN AGENCIA'])]

df_asignacion_test['CODIGO CENTRAL'] = df_asignacion_test['CODIGO CENTRAL'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['N° CONTRATO'] = df_asignacion_test['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_asignacion_test['MONTO A CONDONAR'] = df_asignacion_test['DEUDA CAPITAL'] * df_asignacion_test['% CONDONACIÓN']
df_asignacion_test['TIPO DE FONDO'] = df_asignacion_test['TIPO DE FONDO'].fillna('NULL')
df_asignacion_test['MONTO A PAGAR'] = df_asignacion_test.apply(lambda x: "DEUDA TOTAL" if x['TIPO DE FONDO'] in ['REACTIVA', 'FAE', 'CRECER'] else x['MONTO A PAGAR'], axis=1)

correos = {
    ('ASESCOM RJ', 'BC'): 'karol.lopez@rjabogados.com',
    ('ASESCOM RJ', 'EXJ'): 'lilia.gonzales@rjabogados.com',
    ('ASESCOM RJ', 'KSTBC'): 'rosa.hoyos@rjabogados.com',
    ('CLASA MORA', 'BC'): 'jsalazar@clasaperu.com',
    ('CLASA MORA', 'EXJ'): 'jayra@clasaperu.com',
    ('CLASA MORA', 'KSTBC'): 'jsolis@clasaperu.com',
    ('MORNESE MORA', 'BC'): 'brenda.paz@mornese.pe, juan.mendoza@mornese.pe, informes@mornese.pe',
    ('MORNESE MORA', 'EXJ'): 'aparco@mornese.pe, juan.mendoza@mornese.pe, informes@mornese.pe',
    ('MORNESE MORA', 'KSTBC'): 'juan.mendoza@mornese.pe, geraldine.hinojosa@mornese.pe, informes@mornese.pe',
    ('Muñiz', None): 'szegarra@munizlaw.com',
    ('RodriguezAngobaldo', None): 'ysifuentes@er.com.pe'
}

telefonos = {
    ('ASESCOM RJ', 'BC'): '955013048, 932811608',
    ('ASESCOM RJ', 'EXJ'): '932812578, 932811752',
    ('ASESCOM RJ', 'KSTBC'): '979270273, 932810247, 945309798',
    ('CLASA MORA', 'BC'): '940125238, 977746384',
    ('CLASA MORA', 'EXJ'): '940619571, 977746384',
    ('CLASA MORA', 'KSTBC'): '936994661, 977746384',
    ('MORNESE MORA', 'BC'): '017057808, 953090160',
    ('MORNESE MORA', 'EXJ'): '01-7057893, 977288425',
    ('MORNESE MORA', 'KSTBC'): '017057807, 967140068, 994376233',
    ('Muñiz', None): '999868827',
    ('RodriguezAngobaldo', None): '924104114'
}

def asignar_correo(row):
    return correos.get((row['AGENCIA DE COBRANZA'], row['CARTERA']), correos.get((row['AGENCIA DE COBRANZA'], None), None))

def asignar_telefono(row):
    return telefonos.get((row['AGENCIA DE COBRANZA'], row['CARTERA']), telefonos.get((row['AGENCIA DE COBRANZA'], None), None))

df_asignacion_test['CORREO DE LA AGENCIA'] = df_asignacion_test.apply(asignar_correo, axis=1)
df_asignacion_test['TELÉFONO DE AGENCIA'] = df_asignacion_test.apply(asignar_telefono, axis=1)

df_asignacion_test['CONTACTO BBVA'] = 'recuperaciones@opplus.bbva.com'
df_asignacion_test['CORREO BBVAA'] = 'recuperaciones@opplus.bbva.com'
df_asignacion_test['TELÉFONO'] = '595-0030'

df_asignacion_test.sort_values(by=['CARTERA', 'CODIGO CENTRAL', '% CONDONACIÓN'], inplace=True)
df_asignacion_test.reset_index(drop=True, inplace=True)

cols_final = ['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE', 'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'MONTO A CONDONAR', 'MONTO A PAGAR', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'TELÉFONO DE AGENCIA', 'CORREO DE LA AGENCIA', 'CONTACTO BBVA', 'CORREO BBVAA' , 'TELÉFONO', 'OFICINA', 'TERRITORIO']
df_asignacion_test = df_asignacion_test[cols_final]

print(df_asignacion_test.shape)
df_asignacion_test.head(5)

Index(['codigo', 'contrato', 'cont_18', 'nombre', 'clave', 'producto',
       'DiasVenc', 'Moneda', 'Capital', 'liqtotal', 'agencia', 'Camp',
       'Gestor', 'Tipdoc', 'Numdoc', 'oficina', 'territorio', 'Tramo',
       'codoficina', 'capital_soles', 'CAMPAÑA', 'tipo_fondo', 'foco',
       'flag_honrado', 'ambito', 'TIPO_CARTERA', 'desc_tramo',
       'MOTIVO_INHIBICION', 'Hibrido', 'ESTUDIO ABOGADO', 'FECHA_MORA',
       'ID_CASTIGO', 'ID_VTA.', 'ID_GARANT', 'ID_CONDONACION MORA_TEMP',
       'ID_FRACCIONADO', 'ID_KOBSA_P.CANCELACION', 'ID_REPORTE_CANC',
       'ID_CARTERA', 'ID_FLAG', 'OBSERVACION', 'AGENCIA CORRECTA'],
      dtype='object')
(215149, 18)


,CODIGO CENTRAL,N° CONTRATO,CARTERA,NOMBRE DEL CLIENTE,MONEDA,DEUDA CAPITAL,% CONDONACIÓN,MONTO A CONDONAR,MONTO A PAGAR,AGENCIA DE COBRANZA,TIPO DE FONDO,TELÉFONO DE AGENCIA,CORREO DE LA AGENCIA,CONTACTO BBVA,CORREO BBVAA,TELÉFONO,OFICINA,TERRITORIO
0,00420425,001101639600080207,BC,COMERCIAL ELECTRICAINDUSTRIAL SA,USD,20000.00,0.75,15000.000,5000.0,CLASA MORA,NULL,"940125238, 977746384",jsalazar@clasaperu.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.APOLO,DIVISION LIMA CENTRO
1,01242997,001101139600233442,BC,MIGUEL LUDOWIEG SA,PEN,33727.27,0.00,0.000,DEUDA TOTAL,MORNESE MORA,REACTIVA,"017057808, 953090160","brenda.paz@mornese.pe, juan.mendoza@mornese.pe...",recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.INGENIERIA,DIVISION LIMA NORTE
2,02043432,001103879600179465,BC,MARIA ESTHER VILCA SEGOVIA,PEN,58831.16,0.75,44123.370,14707.79,ASESCOM RJ,NULL,"955013048, 932811608",karol.lopez@rjabogados.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.CANAVAL MOREYRA,DIVISION MIRAFLORES
3,02695081,001102354390002835,BC,GERARDO RUBEN MANCHEGO MOY,PEN,46116.26,0.70,32281.382,13834.878,MORNESE MORA,NULL,"017057808, 953090160","brenda.paz@mornese.pe, juan.mendoza@mornese.pe...",recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.HUANCAYO,DIVISION CENTRO ORIENTE
4,04316835,001102109600686339,BC,HERNAN ALBERTO BAZAN OJEDA,PEN,11133.99,0.00,0.000,DEUDA TOTAL,MORNESE MORA,REACTIVA,"017057808, 953090160","brenda.paz@mornese.pe, juan.mendoza@mornese.pe...",recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.HUANUCO,DIVISION CENTRO ORIENTE


In [4]:
#df_asignacion_test.to_excel('bases/campañas/CAMPAÑAS SEPTIEMBRE 2024.xlsx', index=False)

In [5]:
##########################################################################################################################

In [11]:
base_impago_path = 'bases/campañas/BASE_CONDONACION_202409.xlsx'
df_impago = pd.read_excel(base_impago_path, sheet_name='DATA')
df_impago.shape

(15270, 80)

In [14]:
df_impago_test = df_impago.copy()
print(df_impago_test.columns)

cols_required = ['CODCENTRAL', 'CONTRATO', 'CLIENTE', 'MONEDA', 'DIRECTA', 'TASA_DSCTO', 'MONTOPAGO', 'ENTIDAD_COB', 'OFICINA', 'TERRITORIO', 'SEGMENTO']
df_impago_test = df_impago_test[cols_required]

df_impago_test.rename(columns={
    'CODCENTRAL': 'CODIGO CENTRAL', 
    'CONTRATO': 'N° CONTRATO', 
    'CLIENTE': 'NOMBRE DEL CLIENTE', 
    'DIRECTA': 'DEUDA CAPITAL', 
    'TASA_DSCTO': '% CONDONACIÓN', 
    'MONTOPAGO': 'MONTO A PAGAR', 
    'ENTIDAD_COB': 'AGENCIA DE COBRANZA', 
}, inplace=True)

df_impago_test['CARTERA'] = 'IMPAGO'
df_impago_test['TIPO DE FONDO'] = 'NULL'

df_impago_test['CODIGO CENTRAL'] = df_impago_test['CODIGO CENTRAL'].astype(str).astype('Int64').astype(str).str.zfill(8)
df_impago_test['N° CONTRATO'] = df_impago_test['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_impago_test['MONTO A CONDONAR'] = df_impago_test['DEUDA CAPITAL'] * df_impago_test['% CONDONACIÓN']
df_impago_test['TERRITORIO'] = df_impago_test['TERRITORIO'].fillna('NULL').astype(str)

correos = {
    ('RECSA', 'PARTICULARES'): 'alfonso.moises@recsa.com / jean.yanqui@recsa.com',
    ('RECSA', 'PARTICULARES CONVENIOS'): 'alfonso.moises@recsa.com / jean.yanqui@recsa.com',
    ('RECSA', 'PYME'): 'sandra.clavo@recsa.com / katiuska.vera@recsa.com',
    ('ATENTO', 'PARTICULARES'): 'cvillarj@atento.com',
    ('ATENTO', 'PARTICULARES CONVENIOS'): 'cvillarj@atento.com',
    ('ATENTO', 'PYME'): 'jtumialanp@atento.com',
    ('ASESCOM RJ', 'PARTICULARES'): 'virginia.sifuentes@rjabogados.com',
    ('ASESCOM RJ', 'PARTICULARES CONVENIOS'): 'virginia.sifuentes@rjabogados.com',
    ('ASESCOM RJ', 'PYME'): 'bety.cabrera@rjabogados.com',
}

telefonos = {
    ('RECSA', 'PARTICULARES'): '01 7010519 / 961370088',
    ('RECSA', 'PARTICULARES CONVENIOS'): '01 7010519 / 961370088',
    ('RECSA', 'PYME'): '01 7099302',
    ('ATENTO', 'PARTICULARES'): '996044852 / 990384855 / 967683768',
    ('ATENTO', 'PARTICULARES CONVENIOS'): '996044852 / 990384855 / 967683768',
    ('ATENTO', 'PYME'): '987708366',
    ('ASESCOM RJ', 'PARTICULARES'): '938562012',
    ('ASESCOM RJ', 'PARTICULARES CONVENIOS'): '938562012',
    ('ASESCOM RJ', 'PYME'): '908874634',
}

def asignar_correo(row):
    return correos.get((row['AGENCIA DE COBRANZA'], row['SEGMENTO']), correos.get((row['AGENCIA DE COBRANZA'], None), None))

def asignar_telefono(row):
    return telefonos.get((row['AGENCIA DE COBRANZA'], row['SEGMENTO']), telefonos.get((row['AGENCIA DE COBRANZA'], None), None))


df_impago_test['CORREO DE LA AGENCIA'] = df_impago_test.apply(asignar_correo, axis=1)
df_impago_test['TELÉFONO DE AGENCIA'] = df_impago_test.apply(asignar_telefono, axis=1)

df_impago_test['CONTACTO BBVA'] = 'recuperaciones@opplus.bbva.com'
df_impago_test['CORREO BBVAA'] = 'recuperaciones@opplus.bbva.com'
df_impago_test['TELÉFONO'] = '595-0030'

df_impago_test.sort_values(by=['CARTERA', 'CODIGO CENTRAL', '% CONDONACIÓN'], inplace=True)
df_impago_test.reset_index(drop=True, inplace=True)

cols_final = ['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE', 'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'MONTO A CONDONAR', 'MONTO A PAGAR', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'TELÉFONO DE AGENCIA', 'CORREO DE LA AGENCIA', 'CONTACTO BBVA', 'CORREO BBVAA' , 'TELÉFONO', 'OFICINA', 'TERRITORIO']
df_impago_test = df_impago_test[cols_final]

print(df_impago_test.shape)
df_impago_test.head(5)

Index(['CODCENTRAL', 'tipdoc', 'numdoc', 'CONTRATO', 'TERRITORIO', 'OFICINA',
       'ENTIDAD_COB', 'SEGMENTO', 'ZONA_CLIENTE', 'DEUDATOTAL',
       'CAMINO_APREMIO', 'CLIENTE', 'PRODUCTO', 'MONEDA', 'DIRECTA',
       'CATRASADA', 'DA', 'TRAMO', 'TRAMO_EFECTIVIDAD', 'AFECTACION',
       'MONTOPAGO', 'TASA_DSCTO', 'tiptel1', 'pretel1', 'numtel1', 'exttel1',
       'tiptel2', 'pretel2', 'numtel2', 'exttel2', 'tiptel3', 'pretel3',
       'numtel3', 'exttel3', 'a1tipt1', 'a1pret1', 'a1numt1', 'a1exte1',
       'a1tipt2', 'a1pret2', 'a1numt2', 'a1exte2', 'a2tipt1', 'a2pret1',
       'a2numt1', 'a2exte1', 'a2tipt2', 'a2pret2', 'a2numt2', 'a2exte2',
       'a3tipt1', 'a3pret1', 'a3numt1', 'a3exte1', 'a3tipt2', 'a3pret2',
       'a3numt2', 'a3exte2', 'email', 'tipotelefono', 'prefijo',
       'cargc_ntelefono', 'cargc_nextension', 'FECHACAE', 'MAX_CLASIF_PROY',
       'PROVPROY', 'GASTO_PROY', 'gasto_proy_cliente',
       'gasto_proy_cliente_prorrateado', 'ca_proy', 'CATRASADAHOY',
       'DIR

,CODIGO CENTRAL,N° CONTRATO,CARTERA,NOMBRE DEL CLIENTE,MONEDA,DEUDA CAPITAL,% CONDONACIÓN,MONTO A CONDONAR,MONTO A PAGAR,AGENCIA DE COBRANZA,TIPO DE FONDO,TELÉFONO DE AGENCIA,CORREO DE LA AGENCIA,CONTACTO BBVA,CORREO BBVAA,TELÉFONO,OFICINA,TERRITORIO
0,00006041,001103209600307859,IMPAGO,VICTOR HUGO ABURTO LOROÐA,PEN,76745.68,0.40,30698.2720,46047.4080,RECSA,NULL,01 7010519 / 961370088,alfonso.moises@recsa.com / jean.yanqui@recsa.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.ESSALUD,DIVISION LIMA OESTE
1,00073091,001101955001547147,IMPAGO,LUIS ALBERTO ALZAMORA MENDOZA,PEN,16415.43,0.40,6566.1720,9849.2580,RECSA,NULL,01 7010519 / 961370088,alfonso.moises@recsa.com / jean.yanqui@recsa.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.CTEL GRAL DEL EJERCITO,DIVISION LIMA ESTE
2,00086550,001107855002536944,IMPAGO,PAUL IVAN ANGELES SEVILLA,PEN,3209.91,0.25,802.4775,2407.4325,RECSA,NULL,01 7010519 / 961370088,alfonso.moises@recsa.com / jean.yanqui@recsa.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF.CAMINO REAL,DIVISION MIRAFLORES
3,00117790,001105045003694625,IMPAGO,JAIME ARIAS MEDINA,PEN,3627.55,0.40,1451.0200,2176.5300,RECSA,NULL,01 7010519 / 961370088,alfonso.moises@recsa.com / jean.yanqui@recsa.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF. LA PLANICIE,OFICINA PRIME CRONOS
4,00117790,001102625000187426,IMPAGO,JAIME ARIAS MEDINA,PEN,16636.38,0.40,6654.5520,9981.8280,RECSA,NULL,01 7010519 / 961370088,alfonso.moises@recsa.com / jean.yanqui@recsa.com,recuperaciones@opplus.bbva.com,recuperaciones@opplus.bbva.com,595-0030,OF. LA PLANICIE,OFICINA PRIME CRONOS


In [37]:
#df_impago_test.to_excel('bases/campañas/CAMPAÑAS SEPTIEMBRE 2024_IMPAGO.xlsx', index=False)

In [ ]:
##########################################################################################################################

In [15]:
df_camp_final = pd.concat([df_asignacion_test, df_impago_test])
df_camp_final.shape

(230419, 18)

In [16]:
df_camp_final['CODIGO CENTRAL'] = df_camp_final['CODIGO CENTRAL'].astype(str).astype("Int64").astype(str).str.zfill(8)
df_camp_final['N° CONTRATO'] = df_camp_final['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

In [17]:
df_camp_final.to_excel('bases/campañas/CAMPAÑAS SEPTIEMBRE 2024 FINAL.xlsx', index=False, sheet_name='BASE')